In [1]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
NUM_PLAYERS = 4

In [2]:
def pad_to_length(tensor_list, max_len=7):
    padded = pad_sequence(tensor_list, batch_first=True, padding_value=0)
    if padded.size(1) < max_len:
        pad_size = max_len - padded.size(1)
        padded = F.pad(padded, (0, pad_size), value=0)
    else:
        padded = padded[:, :max_len]
    return padded

# Function to process and pad .npy dataset
def process_round_data(npy_file, max_len=7):
    data = np.load(npy_file)
    x = data[:, :-1]
    y = data[:, -1]
    x_tensor_list = [torch.tensor(seq, dtype=torch.long) for seq in x]
    x_padded = pad_to_length(x_tensor_list, max_len=max_len)
    y_tensor = torch.tensor(y, dtype=torch.float)
    return x_padded, y_tensor

# === Load and process each round ===
x_first, y_first = process_round_data('data_for_first_round_n.npy', max_len=5 + 2 * NUM_PLAYERS)
x_second, y_second = process_round_data('data_for_second_round_n.npy', max_len=5 + 2 * NUM_PLAYERS)
x_third, y_third = process_round_data('data_for_third_round_n.npy', max_len=5 + 2 * NUM_PLAYERS)

# === Concatenate all rounds and repeat them===
x_first_repeated = x_first.repeat((20, 1))        
y_first_repeated = y_first.repeat((20,)) 
x_second_repeated = x_second.repeat((5, 1))        
y_second_repeated = y_second.repeat((5,))           

x_all = torch.cat([x_first_repeated, x_second_repeated, x_third], dim=0)
y_all = torch.cat([y_first_repeated, y_second_repeated, y_third], dim=0)

# === Final dataset and dataloader ===
dataset_all = TensorDataset(x_all, y_all)
dataloader_all = DataLoader(dataset_all, batch_size=32, shuffle=True)

# === Inspect sizes ===
print("First round input shape:", x_first.shape)
print("First round label shape:", y_first.shape)

print("First round repeated input shape:", x_first_repeated.shape)
print("First round repeated label shape:", y_first_repeated.shape)

print("Second round input shape:", x_second.shape)
print("Second round label shape:", y_second.shape)

print("Second round repeated input shape:", x_second_repeated.shape)
print("Second round repeated label shape:", y_second_repeated.shape)

print("Third round input shape:", x_third.shape)
print("Third round label shape:", y_third.shape)

print("Combined input shape:", x_all.shape)
print("Combined label shape:", y_all.shape)

First round input shape: torch.Size([1326, 13])
First round label shape: torch.Size([1326])
First round repeated input shape: torch.Size([26520, 13])
First round repeated label shape: torch.Size([26520])
Second round input shape: torch.Size([19890, 13])
Second round label shape: torch.Size([19890])
Second round repeated input shape: torch.Size([99450, 13])
Second round repeated label shape: torch.Size([99450])
Third round input shape: torch.Size([298350, 13])
Third round label shape: torch.Size([298350])
Combined input shape: torch.Size([424320, 13])
Combined label shape: torch.Size([424320])


In [3]:
print(x_first_repeated)
print(x_second_repeated)
print(x_third)
print(x_all)
print(y_third)

tensor([[ 1,  2, 30,  ...,  0,  0,  0],
        [ 1,  3, 18,  ...,  0,  0,  0],
        [ 1,  4, 50,  ...,  0,  0,  0],
        ...,
        [50, 51, 11,  ...,  0,  0,  0],
        [50, 52, 14,  ...,  0,  0,  0],
        [51, 52, 15,  ...,  0,  0,  0]])
tensor([[ 1,  2, 30,  ..., 12,  0,  0],
        [ 1,  2, 30,  ..., 31,  0,  0],
        [ 1,  2, 30,  ..., 52,  0,  0],
        ...,
        [51, 52, 15,  ..., 40,  0,  0],
        [51, 52, 15,  ..., 21,  0,  0],
        [51, 52, 15,  ..., 49,  0,  0]])
tensor([[ 1,  2, 30,  ..., 12, 10, 44],
        [ 1,  2, 30,  ..., 12, 22, 24],
        [ 1,  2, 30,  ..., 12,  3, 31],
        ...,
        [51, 52, 15,  ..., 49,  8, 21],
        [51, 52, 15,  ..., 49, 27, 40],
        [51, 52, 15,  ..., 49,  4, 38]])
tensor([[ 1,  2, 30,  ...,  0,  0,  0],
        [ 1,  3, 18,  ...,  0,  0,  0],
        [ 1,  4, 50,  ...,  0,  0,  0],
        ...,
        [51, 52, 15,  ..., 49,  8, 21],
        [51, 52, 15,  ..., 49, 27, 40],
        [51, 52, 15,  ...

In [4]:
class EmbeddingNetLinear(nn.Module):
    def __init__(self, num_cards=7, embed_dim=16):
        super(EmbeddingNetLinear, self).__init__()
        self.num_cards = num_cards
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(53, embed_dim, padding_idx=0)

        self.fc1 = nn.Linear(num_cards * embed_dim, 128)  # Increased capacity
        self.fc2 = nn.Linear(128, 64)                     # New layer
        self.fc3 = nn.Linear(64, 24)
        self.fc4 = nn.Linear(24, 1)                       # Output layer

    def forward(self, x):
        embedded = self.embedding(x)  # Shape: [batch_size, num_cards, embed_dim]
        flat = embedded.view(x.size(0), self.num_cards * self.embed_dim)
        out = F.relu(self.fc1(flat))
        out = F.relu(self.fc2(out))
        out = F.relu(self.fc3(out))
        return self.fc4(out).squeeze(1)
    

class EmbeddingNetConv1D(nn.Module):
    def __init__(self, num_cards=7, embed_dim=16):
        super(EmbeddingNetConv1D, self).__init__()
        self.num_cards = num_cards
        self.embed_dim = embed_dim

        self.embedding = nn.Embedding(53, embed_dim, padding_idx=0)

        self.conv1 = nn.Conv1d(in_channels=embed_dim, out_channels=16, kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5)
        

        self.pool = nn.AdaptiveMaxPool1d(1)  # Reduces to [batch, 64, 1]

        self.fc = nn.Linear(32, 1)  # Final output layer

    def forward(self, x):
        x = self.embedding(x)           
        x = x.transpose(1, 2)           
        x = F.relu(self.conv1(x))       
        x = F.relu(self.conv2(x))           
        x = self.pool(x).squeeze(-1)    
        return self.fc(x).squeeze(1)

In [5]:
def train_model(model, dataloader, epochs=10, lr=0.001):

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss() 

    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        correct = 0
        total = 0

        for x_batch, y_batch in dataloader:
            x_batch = x_batch
            y_batch = y_batch

            optimizer.zero_grad()
            preds = model(x_batch)

            loss = criterion(preds, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            predicted = (preds > 0.5).float()
            # Accuracy: prediction within ±tolerance of target
            within_tolerance = (torch.abs(preds - y_batch) <= 0.1).float()
            correct += within_tolerance.sum().item()
            total += y_batch.size(0)

        acc = correct / total
        print(f"Epoch {epoch}/{epochs} | Loss: {total_loss:.4f} | Accuracy: {acc:.4f}")

In [6]:
#linearModel = EmbeddingNetLinear(7, 8)
linearModel = EmbeddingNetLinear(5 + 2 * NUM_PLAYERS, 8)
train_model(linearModel, dataloader_all, 50, 0.001)

Epoch 1/50 | Loss: 147207.3393 | Accuracy: 0.0479
Epoch 2/50 | Loss: 141216.1991 | Accuracy: 0.0532
Epoch 3/50 | Loss: 134814.6296 | Accuracy: 0.0559
Epoch 4/50 | Loss: 134465.8749 | Accuracy: 0.0563
Epoch 5/50 | Loss: 131452.7985 | Accuracy: 0.0566
Epoch 6/50 | Loss: 128790.8243 | Accuracy: 0.0585
Epoch 7/50 | Loss: 126344.9089 | Accuracy: 0.0592
Epoch 8/50 | Loss: 118216.6832 | Accuracy: 0.0589
Epoch 9/50 | Loss: 118521.9982 | Accuracy: 0.0591
Epoch 10/50 | Loss: 110068.1924 | Accuracy: 0.0594
Epoch 11/50 | Loss: 110279.7969 | Accuracy: 0.0584
Epoch 12/50 | Loss: 104251.0020 | Accuracy: 0.0584
Epoch 13/50 | Loss: 101719.7734 | Accuracy: 0.0593
Epoch 14/50 | Loss: 96671.1408 | Accuracy: 0.0594
Epoch 15/50 | Loss: 92475.7178 | Accuracy: 0.0606
Epoch 16/50 | Loss: 87116.4340 | Accuracy: 0.0599
Epoch 17/50 | Loss: 91198.9363 | Accuracy: 0.0595
Epoch 18/50 | Loss: 89786.9559 | Accuracy: 0.0596
Epoch 19/50 | Loss: 82905.0673 | Accuracy: 0.0604
Epoch 20/50 | Loss: 80283.9201 | Accuracy: 0.0

## Get thresholds

In [7]:
print(x_second.shape[0])
print(x_third.shape[0])

num_first = x_first.shape[0]
num_second = x_second.shape[0]
num_third = x_third.shape[0]

# Calculate how many second/third per first
repeat_second_per_first = num_second // num_first
repeat_third_per_second = num_third // num_second
print(num_first)
print(repeat_second_per_first)
print(repeat_third_per_second)
x_all = []
y_all = []

for i in range(num_first):
    x1 = x_first[i]
    y1 = y_first[i]
    
    for j in range(repeat_second_per_first):
        idx2 = i * repeat_second_per_first + j
        x2 = x_second[idx2]
        y2 = y_second[idx2]
        
        for k in range(repeat_third_per_second):
            idx3 = idx2 * repeat_third_per_second + k
            x3 = x_third[idx3]
            y3 = y_third[idx3]
            
            x_all.extend([x1.clone(), x2.clone(), x3.clone()])
            y_all.extend([y1.clone(), y2.clone(), y3.clone()])

x_all_stacked = torch.stack(x_all)  # shape: (num_hands * 3, 7)
y_all_stacked = torch.tensor(y_all, dtype=torch.float)
dataset_all_rounds = TensorDataset(x_all_stacked, y_all_stacked)

19890
298350
1326
15
15


In [8]:
print(x_all_stacked.shape)

torch.Size([895050, 13])


In [9]:
from torch.utils.data import DataLoader
import torch
import pandas as pd

# 1. Create DataLoader
dataloader = DataLoader(dataset_all_rounds, batch_size=264, shuffle=False)

# 2. Set model to eval mode
linearModel.eval()

# 3. Collect everything
all_preds = []
all_inputs = []
all_targets = []

with torch.no_grad():
    for x_batch, y_batch in dataloader:
        preds = linearModel(x_batch)

        all_inputs.append(x_batch)
        all_targets.append(y_batch)
        all_preds.append(preds)

# 4. Concatenate everything
all_inputs = torch.cat(all_inputs, dim=0)    # shape: (N, 7)
all_targets = torch.cat(all_targets, dim=0)  # shape: (N,)
all_preds = torch.cat(all_preds, dim=0)      # shape: (N,)

# 5. Convert to NumPy for use with pandas
inputs_np = all_inputs.cpu().numpy()
targets_np = all_targets.cpu().numpy()
preds_np = all_preds.cpu().numpy()

# 6. Create DataFrame
columns = [f"C{i+1}" for i in range(inputs_np.shape[1])]
df_with_pred = pd.DataFrame(inputs_np, columns=columns)
df_with_pred["y_true"] = targets_np
df_with_pred["y_pred"] = preds_np

In [10]:
print(df_with_pred.head(10))

   C1  C2  C3  C4  C5  C6  C7  C8  C9  C10  C11  C12  C13    y_true    y_pred
0   1   2  30  27  17  48   4  19   0    0    0    0    0 -1.878667 -1.987945
1   1   2  30  27  17  48   4  19  20   39   12    0    0 -2.310000 -1.376410
2   1   2  30  27  17  48   4  19  20   39   12   10   44 -2.350000 -1.039504
3   1   2  30  27  17  48   4  19   0    0    0    0    0 -1.878667 -1.987945
4   1   2  30  27  17  48   4  19  20   39   12    0    0 -2.310000 -1.376410
5   1   2  30  27  17  48   4  19  20   39   12   22   24 -2.400000 -1.846767
6   1   2  30  27  17  48   4  19   0    0    0    0    0 -1.878667 -1.987945
7   1   2  30  27  17  48   4  19  20   39   12    0    0 -2.310000 -1.376410
8   1   2  30  27  17  48   4  19  20   39   12    3   31 -2.650000 -1.026367
9   1   2  30  27  17  48   4  19   0    0    0    0    0 -1.878667 -1.987945


In [11]:
df_with_pred["round"] = df_with_pred.index % 3  # 0, 1, 2 repeating
df_with_pred["hand_id"] = df_with_pred.index // 3
# Set random seed for reproducibility
np.random.seed(42)

# Get unique hand_ids
unique_hands = df_with_pred["hand_id"].unique()

# Sample a % of them
sampled_hands = np.random.choice(unique_hands, size=int(0.5 * len(unique_hands)), replace=False)

# Filter the DataFrame to keep only those hands
df_sample = df_with_pred[df_with_pred["hand_id"].isin(sampled_hands)].copy()

In [12]:
def evaluate_thresholds_fast(df, thresholds):
    # Extract as numpy arrays for speed
    y_pred = df["y_pred"].values
    y_true = df["y_true"].values
    rounds = df["round"].values
    hand_ids = df["hand_id"].values

    num_hands = hand_ids[-1] + 1  # assumes hand_ids are 0-based consecutive

    # Pre-allocate returns and bets
    total_return = 0
    total_bet = 0

    i = 0
    while i < len(y_pred):
        # Process 3 rows at a time: preflop (0), flop (1), river (2)
        r = rounds[i:i+3]
        yp = y_pred[i:i+3]
        yt = y_true[i:i+3]

        for j in range(3):
            if yp[j] >= thresholds[r[j]]:
                total_return += yt[j]
                total_bet += 6 if r[j] == 0 else 4 if r[j] == 1 else 3
                break
        else:
            total_return += -2 
            total_bet += 2 # fold penalty

        i += 3  # jump to next hand

    return total_return / total_bet if total_bet > 0 else float('-inf')

In [13]:
from scipy.optimize import minimize
import numpy as np

# Objective function: negative of your custom evaluate_thresholds
def objective(threshold_array):
    thresholds = {0: threshold_array[0], 1: threshold_array[1], 2: threshold_array[2]}
    print(f"Evaluating thresholds: {thresholds}")
    return -evaluate_thresholds_fast(df_sample, thresholds)

# Initial guess (you can tweak this)
initial_guess = [-0.5, -0.5, -0.5]

# Optional: bounds for each threshold
bounds = [(-2, 2), (-2, 2), (-3, 0)]

result = minimize(
    objective,
    initial_guess,
    method='Powell',  # or 'Powell' or 'L-BFGS-B'
    bounds=bounds,
    options={'disp': True, 'maxiter': 200}
)

# Extract results
best_thresholds = {i: t for i, t in enumerate(result.x)}
max_return = -result.fun

print("\n✅ Optimization complete.")
print("Best thresholds:", best_thresholds)
print("Max return:", max_return)

Evaluating thresholds: {0: np.float64(-0.5), 1: np.float64(-0.5), 2: np.float64(-0.5)}
Evaluating thresholds: {0: np.float64(-0.4721359549995796), 1: np.float64(-0.5), 2: np.float64(-0.5)}
Evaluating thresholds: {0: np.float64(0.47213595499957917), 1: np.float64(-0.5), 2: np.float64(-0.5)}
Evaluating thresholds: {0: np.float64(-1.0557280900008412), 1: np.float64(-0.5), 2: np.float64(-0.5)}
Evaluating thresholds: {0: np.float64(-1.4018921318065316), 1: np.float64(-0.5), 2: np.float64(-0.5)}
Evaluating thresholds: {0: np.float64(-1.2398412122880638), 1: np.float64(-0.5), 2: np.float64(-0.5)}
Evaluating thresholds: {0: np.float64(-1.244386765312131), 1: np.float64(-0.5), 2: np.float64(-0.5)}
Evaluating thresholds: {0: np.float64(-1.2347466326361163), 1: np.float64(-0.5), 2: np.float64(-0.5)}
Evaluating thresholds: {0: np.float64(-1.1663676339659133), 1: np.float64(-0.5), 2: np.float64(-0.5)}
Evaluating thresholds: {0: np.float64(-1.1983302925737507), 1: np.float64(-0.5), 2: np.float64(-0.

## Testing model 

In [14]:
import random
# define card set
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']
ranks = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A']
rank_values = {rank: i for i, rank in enumerate(ranks, start=2)}

deck = [{'rank': rank, 'suit': suit} for suit in suits for rank in ranks]

combinations = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Four of a Kind", 
                "Full House", "Straight", "Flush", "Straight Flush", "Royal Flush"]
combinations_values = {combination: i for i, combination in enumerate(combinations, start=1)}
# set ordered winning combinations
winning_hands = ["High Card", "One Pair", "Two Pair", "Three of a Kind", "Straight", "Flush", 
                "Full House", "Four of a Kind", "Straight Flush", "Royal Flush"]

winning_hand_ranks = {hand: i for i, hand in enumerate(winning_hands)}
#enumerate the deck
enumerated_deck = dict(enumerate(deck, start=1))
num_deck = list(range(1, 53))     

In [15]:
def get_model_input_based_on_round(cards, round):
    if round == 0:
        input_cards = cards[:2 * NUM_PLAYERS]
    elif round == 1:
        input_cards = cards[:3 + 2 * NUM_PLAYERS]
    else:
        input_cards = cards[:5 + 2 * NUM_PLAYERS]
    
    # Convert to tensor
    input_tensor = torch.tensor(input_cards, dtype=torch.long)

    # Pad with zeros on the right if needed
    if len(input_tensor) < 5 + 2 * NUM_PLAYERS:
        pad_size = (5 + 2 * NUM_PLAYERS) - len(input_tensor)
        input_tensor = F.pad(input_tensor, (0, pad_size), value=0)
    else:
        input_tensor = input_tensor[:5 + 2 * NUM_PLAYERS]  # Truncate just in case

    return input_tensor.unsqueeze(0)  # shape: (max_len,)

In [16]:
def count_intervals(data, interval_size, min_val=-5, max_val=5):
    bins = {}
    num_bins = int((max_val - min_val) / interval_size)

    for value in data:
        if value < min_val or value >= max_val:
            continue  # skip out-of-range values

        # shift the value range to start at 0
        bin_index = int((value - min_val) / interval_size)
        bin_start = round(min_val + bin_index * interval_size, 10)
        bin_end = round(bin_start + interval_size, 10)

        bin_label = f"[{bin_start}, {bin_end})"
        if bin_label not in bins:
            bins[bin_label] = 0
        bins[bin_label] += 1

    return bins

In [17]:
import ultimate
def test_model_with_games(model, thresholds=[0, 0, 0], num_games = 100):
    model.eval()
    budget = 0
    allBet = 0
    folds = 0
    flops = 0
    rivers = 0
    preflops = 0
    
    # list of predictions
    preds = []
    for i in range(num_games): 
        # generate game (9 cards, played, river, dealer)
        cards = random.sample(num_deck, 7 + 2 * NUM_PLAYERS)
        winnings = 0

        # check what round we are in
        round = 0
        round_when_bet = None
        # play  game until it ends
        while True:
            model_input = get_model_input_based_on_round(cards, round)
            
            # Forward pass to get prediction (probability of betting 1)
            with torch.no_grad():
                pred = model(model_input)  # shape: [1, 1]

            pred_prob = pred.item()  # get scalar probability
            preds.append(pred_prob)
            # if prediction > 0.5 => bet, else don't bet
            if pred_prob > thresholds[round] and round_when_bet is None:
                round_when_bet = round
                break

            round += 1

            if round == 3:
                break

        # calculate winnings
        player_hand = [enumerated_deck[card] for card in cards[0:2]] + [enumerated_deck[card] for card in cards[2*NUM_PLAYERS:5+2*NUM_PLAYERS]]

        dealer_hand = [enumerated_deck[card] for card in cards[2*NUM_PLAYERS:]]
        
        player_combination = ultimate.get_best_hand(player_hand)
        dealer_combination = ultimate.get_best_hand(dealer_hand)

        player_rank = winning_hand_ranks[player_combination]
        dealer_rank = winning_hand_ranks[dealer_combination]

        victor = 0 # 0 = dealer, 1 = player
        
        if player_rank > dealer_rank:
            victor = 1
        elif player_rank == dealer_rank:
            result = ultimate.decider(player_combination, player_hand, 
                                    dealer_combination, dealer_hand)
            if result == "player":
                victor = 1
            elif result == "dealer":	
                victor = 0
            else:
                victor = 2	# need to decide about this
                winnings = 0
        else:
            victor = 0

        # check if ante is valid
        dealer_has_something = ultimate.dealer_has_pair_or_better(dealer_hand[:2], dealer_hand[2:])
        blind_won = ultimate.has_blind(1, player_combination) - 1 #how much blind got us

        # calculate rewards for first and second rounds (in third victory is already bet, defeat is fold)
        if round_when_bet == 0:
            if victor == 1:
                winnings =  4 + blind_won + (1 if dealer_has_something else 0)
            elif victor == 0:
                winnings =  -6 + (1 if not dealer_has_something else 0)
            allBet += 6
            preflops += 1
        elif round_when_bet == 1:
            if victor == 1:
                winnings =  2 + blind_won + (1 if dealer_has_something else 0)
            elif victor == 0:
                winnings =  -4 + (1 if not dealer_has_something else 0)
            allBet += 4
            flops += 1
        elif round_when_bet == 2:
            if victor == 1:
                winnings =  1 + blind_won + (1 if dealer_has_something else 0)
            elif victor == 0:
                winnings =  -3 + (1 if not dealer_has_something else 0)
            allBet += 3
            rivers += 1
        elif round_when_bet == None:
            winnings = -2
            allBet += 2
            folds += 1
        
 
        budget += winnings
        #print("Winnings: ", winnings)
    
    print("Budget is: ", budget)
    print("Betted: ", allBet)
    print("PreFlops: ", preflops)
    print("Flops:", flops)
    print("Rivers:", rivers)
    print("Folds: ", folds)
    print("Ratio: ", budget / allBet)
    #print(preds)
    interval_size = 0.2
    result = count_intervals(preds, interval_size)
    for interval, count in sorted(result.items()):
        print(f"{interval}: {count}")

In [18]:
test_model_with_games(linearModel, best_thresholds,  1000000)

Budget is:  -244759.5
Betted:  4582948
PreFlops:  489257
Flops: 144541
Rivers: 336838
Folds:  29364
Ratio:  -0.05340656276265845
[-0.2, 0.0): 95235
[-0.4, -0.2): 113316
[-0.6, -0.4): 133269
[-0.8, -0.6): 150471
[-1.0, -0.8): 159292
[-1.2, -1.0): 157052
[-1.4, -1.2): 143945
[-1.6, -1.4): 123679
[-1.8, -1.6): 99820
[-2.0, -1.8): 75688
[-2.2, -2.0): 54022
[-2.4, -2.2): 35936
[-2.6, -2.4): 21920
[-2.8, -2.6): 12841
[-3.0, -2.8): 6997
[-3.2, -3.0): 3563
[-3.4, -3.2): 1686
[-3.6, -3.4): 835
[-3.8, -3.6): 343
[-4.0, -3.8): 151
[-4.2, -4.0): 57
[-4.4, -4.2): 16
[-4.6, -4.4): 8
[-4.8, -4.6): 4
[-5.0, -4.8): 1
[0.0, 0.2): 80388
[0.2, 0.4): 68927
[0.4, 0.6): 58619
[0.6, 0.8): 50577
[0.8, 1.0): 43310
[1.0, 1.2): 36484
[1.2, 1.4): 30187
[1.4, 1.6): 24940
[1.6, 1.8): 20332
[1.8, 2.0): 16105
[2.0, 2.2): 12593
[2.2, 2.4): 9794
[2.4, 2.6): 7537
[2.6, 2.8): 5654
[2.8, 3.0): 4382
[3.0, 3.2): 3420
[3.2, 3.4): 2636
[3.4, 3.6): 2056
[3.6, 3.8): 1609
[3.8, 4.0): 1217
[4.0, 4.2): 1010
[4.2, 4.4): 755
[4.4, 4.